In [61]:
import pandas as pd
import numpy as np

from sklearn.metrics.pairwise import pairwise_distances

#For plotting
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import datetime as dt

In [62]:
# Get time series data:
data = pd.read_csv('GamesDataAll.csv')

# Make percentile into number and clear "-"
data = data.replace(to_replace='%', value='', regex=True).replace({'-':'NaN'})#.replace(to_replace='Inf', value=+100, regex=True)
#data['Perc_Gain'] = data['Perc_Gain']/100

data.head()

,App_id,Name,Avg_Players,Gain,Perc_Gain,Peak_Players,Last 30 Days,April 2021,March 2021,February 2021,...,April 2013,March 2013,February 2013,January 2013,December 2012,November 2012,October 2012,September 2012,August 2012,July 2012
0,730,Counter-Strike: Global Offensive,1,0,0,0,695044.47,723346.52,740927.82,741013.24,...,16006.0,15959.89,17372.73,16164.22,14079.39,14134.1,10739.48,16000.75,15475.39,932.57
1,730,Counter-Strike: Global Offensive,0,1,0,0,-28302.0,-17581.31,-85.42,-2196.42,...,46.11,-1412.84,1208.51,2084.83,-54.71,3394.62,-5261.27,525.36,14542.81,NaN
2,730,Counter-Strike: Global Offensive,0,0,1,0,-3.91,-2.37,-0.01,-0.30,...,+0.29,-8.13,+7.48,+14.81,-0.39,+31.61,-32.88,+3.39,+1559.43,NaN
3,730,Counter-Strike: Global Offensive,0,0,0,1,1119102,1148077,1198581,1123485,...,31966,32166,35238,31359,27553,50533,20850,36057,52261,1642
4,570,Dota 2,1,0,0,0,413025.27,413784.97,390412.76,404832.13,...,174835.73,180878.9,166972.97,147748.14,121924.42,101077.43,75965.44,61867.68,55768.61,52721.05


In [63]:
# Get Avg_Players.
avg_players = data.loc[data['Avg_Players']==1]
avg_players = avg_players.drop(columns=['Avg_Players', 'Gain', 'Perc_Gain', 'Peak_Players','Last 30 Days'])
avg_pure = avg_players.drop(columns=['App_id', 'Name']).astype(float)

avg_players.sample(5)

,App_id,Name,April 2021,March 2021,February 2021,January 2021,December 2020,November 2020,October 2020,September 2020,...,April 2013,March 2013,February 2013,January 2013,December 2012,November 2012,October 2012,September 2012,August 2012,July 2012
23832,31200,Tales of Monkey Island: Chapter 4 - The Trial ...,2.36,3.64,2.88,3.93,3.35,2.57,2.55,2.26,...,2.34,2.01,2.84,4.0,3.62,3.3,3.14,4.2,5.0,8.59
8908,679900,Swords & Souls: Neverseen,39.15,41.69,54.1,60.49,65.62,46.2,42.53,63.73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9392,55140,MX vs. ATV Reflex,42.17,47.52,53.24,53.57,48.94,45.19,41.08,37.26,...,41.45,57.85,69.36,98.03,76.96,51.83,52.69,46.62,47.14,51.92
30660,1025020,Wondershare Video Converter Ultimate,1.85,1.7,2.21,1.35,1.74,1.79,1.55,1.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21792,214150,Galactic Civilizations I: Ultimate Edition,4.15,3.72,3.53,3.45,3.25,2.79,3.2,3.46,...,5.99,8.14,9.6,26.06,16.94,3.55,6.84,10.74,18.31,NaN


In [64]:
# Add a column of max and sort the dataframe
avg_players['max'] = avg_pure.max(axis=1)
avg_players.sort_values(by=['max'], ascending=False, inplace=True)#.reset_index(drop=True)

avg_graded = avg_players.loc[avg_players['max']>=1].copy()

In [65]:
# Rank the games according to 'max'
ranks = [0,100,500,2000,5000]

print('Using maximum of Avg_Players:')
for i in ranks:
    print('The cutoff for top '+str(i)+' games is '+str(avg_graded.reset_index(drop=True)['max'][i]))

Using maximum of Avg_Players:
The cutoff for top 0 games is 1584886.77
The cutoff for top 100 games is 24820.66
The cutoff for top 500 games is 3291.5
The cutoff for top 2000 games is 395.07
The cutoff for top 5000 games is 49.52


In [66]:
cutoffs = [1] + [avg_graded.reset_index(drop=True)['max'][i] for i in ranks[::-1]]

counts  = avg_graded['max'].value_counts(bins=cutoffs, sort=False,dropna=False) 

#plt.plot(bins[:-1],counts)

print(counts)

(0.999, 49.52]            4128
(49.52, 395.07]           3000
(395.07, 3291.5]          1500
(3291.5, 24820.66]         400
(24820.66, 1584886.77]     100
Name: max, dtype: int64


In [67]:
grades = [1]*100+[2]*400+[3]*1500+[4]*3000+[5]*4128
#avg_graded = avg_players.loc[avg_players['max']>=1].copy()
avg_graded['Grade'] = grades

print('There are '+str(len(grades))+' games stored and assigned with a grade, based on the maximum of Avg_Players.')

There are 9128 games stored and assigned with a grade, based on the maximum of Avg_Players.


In [53]:
# Save the grades of selected 9128 games.
avg_graded.reset_index(drop=True).to_csv('GamesDataRanked.csv')

In [70]:
avg_graded

,App_id,Name,April 2021,March 2021,February 2021,January 2021,December 2020,November 2020,October 2020,September 2020,...,February 2013,January 2013,December 2012,November 2012,October 2012,September 2012,August 2012,July 2012,max,Grade
8,578080,PLAYERUNKNOWN'S BATTLEGROUNDS,186110.65,193114.18,198957.52,201247.19,189233.58,179520.26,162585.64,169093.71,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1584886.77,1
0,730,Counter-Strike: Global Offensive,723346.52,740927.82,741013.24,743209.66,717803.75,668754.58,613666.69,606850.32,...,17372.73,16164.22,14079.39,14134.1,10739.48,16000.75,15475.39,932.57,857604.22,1
4,570,Dota 2,413784.97,390412.76,404832.13,432671.65,422552.32,425904.83,406361.36,408609.78,...,166972.97,147748.14,121924.42,101077.43,75965.44,61867.68,55768.61,52721.05,709178.26,1
252,1091500,Cyberpunk 2077,12994.52,13145.84,24704.84,82146.66,332395.65,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,332395.65,1
200,377160,Fallout 4,12609.86,12642.67,13773.15,15083.94,14459.36,16615.12,15057.58,12872.22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,226297.83,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36708,1051210,Sageball,0.34,1.01,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.01,5
37968,1414480,Zof,0.46,1.0,0.94,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,5
33264,709670,Clarent Saga: Tactics,0.68,0.94,0.93,0.74,0.73,0.85,0.49,0.62,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,5
32000,773310,Spook Inspectors Investigate Morleyville Mall ...,0.83,0.97,1.0,0.97,0.99,1.0,0.99,0.96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00,5
